Initialization

In [10]:
import gym
import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import time
import scipy.signal as signal
import os
import pickle
import imageio

#import sapp_gym
from ACNet_pytorch import ACNet
from ACNet_pytorch import ACNetLoss
#from torchviz import make_dot
import AgentState
from FireFightingEnv import FFEnv

#from torch.utils.tensorboard import SummaryWriter

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
#model = 
#model = model.to(device)


def make_gif(images, fname, duration=2, true_image=False,salience=False,salIMGS=None):
    imageio.mimwrite(fname,images,subrectangles=True)
    print("wrote gif")

def discount(x, gamma):
    return signal.lfilter([1], [1, -gamma], x[::-1], axis=0)[::-1]

cpu


Traning Parameters

In [11]:
# Learning parameters
max_episode_length     = 128
episode_count          = 0
EPISODE_START          = episode_count
gamma                  = .95 # discount rate for advantage estimation and reward discounting
#moved network parameters to ACNet.py
EXPERIENCE_BUFFER_SIZE = 128
GRID_SIZE              = 11 #the size of the FOV grid to apply to each agent
ENVIRONMENT_SIZE       = (15,15)#the total size of the environment (length of one side)
OBSTACLE_DENSITY       = (0,.3) #range of densities
DIAG_MVMT              = False # Diagonal movements allowed?
a_size                 = 4 + 1 + 4 + 1 # action size: 4 direction movements, stop, 4 direction spraying, and go back to water supply station.
SUMMARY_WINDOW         = 10
LR_Q                   = 8.e-5
load_model             = False
RESET_TRAINER          = False
model_path             = 'model_sapp_vanilla'
gifs_path              = 'gifs_sapp_vanilla'
train_path             = 'train_sapp_vanilla'
GLOBAL_NET_SCOPE       = 'global'
agent_num              = 3

# Simulation options
FULL_HELP              = False
OUTPUT_GIFS            = True
SAVE_EPISODE_BUFFER    = False

# Testing
TRAINING               = True
GREEDY                 = False
NUM_EXPS               = 100
MODEL_NUMBER           = 140500

# Shared arrays for tensorboard
episode_rewards        = []
episode_lengths        = []
episode_mean_values    = []
episode_invalid_ops    = []
rollouts               = None
printQ                 = False # (for headless)

Work Agent

In [12]:
class Worker:
    def __init__(self, game, global_network):
        self.env = game
        self.global_network = global_network

        self.nextGIF = episode_count # For GIFs output

    def train(self, rollout, gamma, bootstrap_value, rnn_state0, imitation=False):
        #tf:(self, rollout, sess, gamma, bootstrap_value, rnn_state0, imitation=False)
        global episode_count

        # [s[0],s[1],a,r,s1,d,v[0,0],train_valid]
        rollout     = np.array(rollout)
        observ      = torch.from_numpy(rollout[:,0])
        #goal_vec    = torch.from_numpy(rollout[:,1])
        actions     = torch.from_numpy(rollout[:,2])
        rewards     = torch.from_numpy(rollout[:,3])
        policy      = torch.from_numpy(rollout[:,4])
        values      = torch.from_numpy(rollout[:,6])
        #valids      = torch.from_numpy(rollout[:,7])

        # Here we take the rewards and values from the rollout, and use them to 
        # generate the advantage and discounted returns. (With bootstrapping)
        # The advantage function uses "Generalized Advantage Estimation"
        self.rewards_plus = torch.cat((rewards, bootstrap_value.unsqueeze(0)))
        #self.rewards_plus = np.asarray(rewards.tolist() + [bootstrap_value])
        discounted_rewards = discount(self.rewards_plus,gamma)[:-1]
        self.value_plus = torch.cat((values, bootstrap_value.unsqueeze(0)))
        #self.value_plus = np.asarray(values.tolist() + [bootstrap_value])
        advantages = rewards + gamma * self.value_plus[1:] - self.value_plus[:-1]
        advantages = discount(advantages,gamma)

        # Generate network statistics to periodically save
        #feed_dict = {
        #    self.global_network.target_v:np.stack(discounted_rewards),
        #    self.global_network.inputs:np.stack(observ),
        #    self.global_network.goal_pos:np.stack(goal_vec),
        #    self.global_network.actions:actions,
        #    self.global_network.train_valid:np.stack(valids),
        #    self.global_network.advantages:advantages,
        #    self.global_network.state_in[0]:rnn_state0[0],
        #    self.global_network.state_in[1]:rnn_state0[1]
        #}
 
        ## Modifing grad
        
        target_v = torch.stack(discounted_rewards)
        #inputs = torch.stack(observ)
        # goal_pos = torch.stack(goal_vec)
        train_valid = torch.stack(valids)
        rnn_state = (rnn_state0[0], rnn_state0[1])
        # ACNetLoss.forward(self, policy, value, actions, target_v, advantages)
        total_loss = self.global_network.ACNetLoss.forward(policy, values, actions, target_v, advantages)
        ## Update the global network with gradients 
        #policy, value, state_out, policy_sig = self.global_network.optimizer.step(inputs, water_res)?




        # 以下步骤应该是在learning_agent当中使用
        # Get gradients from local network using local losses and
        # normalize the gradients using clipping
        GRAD_CLIP = gamma
        trainable_vars = torch.from_numpy(policy) # FIXME how to express?
        gradients = self.global_network.torch.autograd.grad(total_loss, trainable_vars, create_graph=True)
        var_norms = self.global_network.torch.norm(torch.cat([v.view(-1) for v in trainable_vars]))
        grad_norms = self.global_network.torch.nn.utils.clip_grad_norm_(gradients, GRAD_CLIP)
        apply_grads = self.global_network.trainer(self.global_network.parameters(), LR_Q)


        return total_loss
    
    def shouldRun(self, episode_count):

        return (episode_count < NUM_EXPS)
    
    def work(self, max_episode_length, gamma):
    #(self, max_episode_length, gamma, sess, coord, saver):
        global episode_count, episode_rewards, episode_lengths, episode_mean_values, episode_invalid_ops
        total_steps, i_buf = 0, 0    

        

 

        while self.shouldRun(episode_count):
                episode_buffer, episode_values = [], []
                episode_reward = episode_step_count = episode_inv_count = 0
                d = False

                # Initial state from the environment for each time runs work()
                s                     = self.env.reset()  #observe_space
                rnn_state             = self.global_network.weights_init() #state init does not have output TODO
                rnn_state0            = rnn_state

                saveGIF = False

                # Initial random action

                for i in range(agent_num):
                    a[0,i] = np.random.choice([0, 1, 3, 3, 4, 5])
                    a[1,i] = np.random.choice([0, 1, 2, 3, 4, 5])

                if OUTPUT_GIFS and ((not TRAINING) or (episode_count >= self.nextGIF)):
                    saveGIF = True
                    self.nextGIF = episode_count + 256
                    GIF_episode = int(episode_count)
                    episode_frames = [ self.env._render(mode='rgb_array',screen_height=900,screen_width=900) ]
                    
                while True: 

                    ##Take an action from network according to current obs.
                    #~ , v, rnn_state, ~ = self.global_network.optimize_model()
                    #obs = np.array(self.env.observe_space)
                    #inputs_raw = np.array(obs[:, :4])
                    #inputs = []
                    #for map in inputs_raw:
                    #    reshaped_map = [np.array(arr) for arr in map]
                    #    inputs.append(reshaped_map)
                    obs = np.array(self.env.observe_space)
                    ##TODO get actions from obs?
                    #ACNet return policy, value, state_out, policy_sig
                    inputs_raw = np.array(obs[:, :4])
                    inputs = []
                    for map in inputs_raw:
                        reshaped_map = [np.array(arr) for arr in map]
                        inputs.append(reshaped_map)
                    inputs = np.array(inputs)
                    water_res = np.array(obs[:, 4]).reshape(-1,1)
                    # run the net.forward to return self.policy, self.value, self.state_out, policy_sig
                    policy, _, v,_ =  self.global_network.forward(inputs,water_res)
                    actions = policy


                    #environment operation and update
                    s1, r, d = self.env.step(actions)
                    #env.step will return self.observe_space, self.rewards, self.finished
                          
                    if saveGIF:
                        episode_frames.append(self.env._render(mode='rgb_array',screen_width=900,screen_height=900))

                    #episode_buffer.append([s[0],s[1],a,r,s1,d,v[0,0],train_valid])
                    #episode_values.append(v[0,0])
                    episode_buffer.append([s[0], s[1],actions, r, s1, d, v.data.cpu().numpy()[0, 0]])
                    episode_values.append(v.data.cpu().numpy()[0, 0])
                    episode_reward += r
                    s = s1
                    total_steps += 1
                    episode_step_count += 1

                    if d == True:
                        print('\n{} Goodbye World. We did it!'.format(episode_step_count), end='\n')

                    ## If the episode hasn't ended, but the experience buffer is full, then we
                    ## make an update step using that experience rollout.
                    if TRAINING and (len(episode_buffer) % EXPERIENCE_BUFFER_SIZE == 0 or d):
                        ## Since we don't know what the true final return is, we "bootstrap" from our current value estimation.
                        if len(episode_buffer) >= EXPERIENCE_BUFFER_SIZE:
                            episode_buffer_training = episode_buffer[-EXPERIENCE_BUFFER_SIZE:]
                        else:
                            episode_buffer_training = episode_buffer[:]

                        if d:
                            s1Value = 0
                        else:
                            s1Value, _ = self.global_network(
                                torch.from_numpy(s[0]), 
                                torch.from_numpy(s[1]), 
                                rnn_state)
                        
                        goal_vec = torch.from_numpy([])
                        rollout     = np.array(s,goal_vec,actions,r,policy,episode_values)
                        
                        #v_l, p_l, valid_l, e_l, g_n, v_n = self.train(
                        #train() will return policy, value, state_out, policy_sig
                        ## update neural network weights in train()
                        total_loss = self.train(
                            episode_buffer_training, rollout, gamma, s1Value, rnn_state0)
                        
                        

                        rnn_state0 = rnn_state

                    if episode_step_count >= max_episode_length or d:
                        break

                episode_lengths.append(episode_step_count)
                episode_mean_values.append(np.nanmean(episode_values))
                episode_invalid_ops.append(episode_inv_count)
                episode_rewards.append(episode_reward)


                if not TRAINING:
                    episode_count += 1
                    print('({}) Thread 0: {} steps, {:.2f} reward ({} invalids).'.format(episode_count, episode_step_count, episode_reward, episode_inv_count))
                    GIF_episode = int(episode_count)
                else:
                    episode_count += 1

                    if episode_count % SUMMARY_WINDOW == 0:
                        if episode_count % 100 == 0:
                            print ('Saving Model', end='\n')
                        #    saver.save(sess, model_path+'/model-'+str(int(episode_count))+'.cptk')
                            torch.save(self.global_network.state_dict(),
                                   os.path.join(model_path, 'model-{}.pth'.format(int(episode_count)))
                                   )
                            print ('Saved Model', end='\n')
                        mean_reward = np.nanmean(episode_rewards[-SUMMARY_WINDOW:])
                        mean_length = np.nanmean(episode_lengths[-SUMMARY_WINDOW:])
                        mean_value = np.nanmean(episode_mean_values[-SUMMARY_WINDOW:])
                        mean_invalid = np.nanmean(episode_invalid_ops[-SUMMARY_WINDOW:])

                        ## Visualisation
                        #summary = tf.Summary()
                        #summary.value.add(tag='Perf/Reward', simple_value=mean_reward)
                        #summary.value.add(tag='Perf/Length', simple_value=mean_length)
                        #summary.value.add(tag='Perf/Valid Rate', simple_value=(mean_length-mean_invalid)/mean_length)

                        #summary.value.add(tag='Losses/Value Loss', simple_value=v_l)
                        #summary.value.add(tag='Losses/Policy Loss', simple_value=p_l)
                        #summary.value.add(tag='Losses/Valid Loss', simple_value=valid_l)
                        #summary.value.add(tag='Losses/Grad Norm', simple_value=g_n)
                        #summary.value.add(tag='Losses/Var Norm', simple_value=v_n)
                        #global_summary.add_summary(summary, int(episode_count))

                        #global_summary.flush()

                        #if printQ:
                        #    print('{} Tensorboard updated'.format(episode_count), end='\r')

                if saveGIF:
                    # Dump episode frames for external gif generation (otherwise, makes the jupyter kernel crash)
                    time_per_step = 0.1
                    images = np.array(episode_frames)
                    if TRAINING:
                        make_gif(images, '{}/episode_{:d}_{:d}_{:.1f}.gif'.format(gifs_path,GIF_episode,episode_step_count,episode_reward))
                    else:
                        make_gif(images, '{}/episode_{:d}_{:d}.gif'.format(gifs_path,GIF_episode,episode_step_count), duration=len(images)*time_per_step,true_image=True,salience=False)
                if SAVE_EPISODE_BUFFER:
                    with open('gifs3D/episode_{}.dat'.format(GIF_episode), 'wb') as file:
                        pickle.dump(episode_buffer, file)

SyntaxError: invalid syntax (<ipython-input-12-5b8f55391b81>, line 127)

Training

In [ ]:
#tf.reset_default_graph()
#print("Hello World")

if not os.path.exists(model_path):
    os.makedirs(model_path)
#config = tf.ConfigProto(allow_soft_placement = True)
#config.gpu_options.allow_growth=True

if not TRAINING:
    gifs_path += '_tests'
    if SAVE_EPISODE_BUFFER and not os.path.exists('gifs3D'):
        os.makedirs('gifs3D')

#Create a directory to save episode playback gifs to
if not os.path.exists(gifs_path):
    os.makedirs(gifs_path)
'''
#with tf.device("/gpu:0"): # uncomment to run on GPU, and comment next line
with tf.device("/cpu:0"):
    trainer = tf.contrib.opt.NadamOptimizer(learning_rate=LR_Q, use_locking=True)

    net = ACNet(GLOBAL_NET_SCOPE, a_size, trainer, True, GRID_SIZE) # Generate global network

    gameEnv = sapp_gym.SAPPEnv(DIAGONAL_MOVEMENT=DIAG_MVMT, SIZE=ENVIRONMENT_SIZE, 
                               observation_size=GRID_SIZE, PROB=OBSTACLE_DENSITY)

    worker  = Worker(gameEnv, net)

    global_summary = tf.summary.FileWriter(train_path)
    saver = tf.train.Saver(max_to_keep=2)

    with tf.Session(config=config) as sess:
        sess.run(tf.global_variables_initializer())
        coord = tf.train.Coordinator()

        if load_model == True:
            print ('Loading Model...')
            if not TRAINING:
                with open(model_path+'/checkpoint', 'w') as file:
                    file.write('model_checkpoint_path: "model-{}.cptk"'.format(MODEL_NUMBER))
                    file.close()
            ckpt = tf.train.get_checkpoint_state(model_path)
            p=ckpt.model_checkpoint_path
            p=p[p.find('-')+1:]
            p=p[:p.find('.')]
            if TRAINING:
                episode_count = int(p)
            else:
                episode_count = 0
            saver.restore(sess,ckpt.model_checkpoint_path)
            print("episode_count set to ",episode_count)
            if RESET_TRAINER:
                trainer = tf.contrib.opt.NadamOptimizer(learning_rate=lr, use_locking=True)

        worker.work(max_episode_length, gamma, sess, coord,saver)
'''
## Create your PyTorch model and optimizer

trainer = torch.optim.SGD
# ACNet.__init__(self, a_size, batch_size, trainer, learning_rate, TRAINING, GRID_SIZE)
net = ACNet(a_size, agent_num, trainer, LR_Q, TRAINING, GRID_SIZE)

game_env = FFEnv(agent_num=agent_num)
worker = Worker(game_env, net)
global_summary = None  ## TODO Set up Tensorboard writer
saver = None  ## TODO Set up PyTorch model saver


if load_model:
    print('Loading Model...')
    if not TRAINING:
        # Load PyTorch model
        model_path = os.path.join(model_path, 'model-{}.pth'.format(MODEL_NUMBER))
        checkpoint = torch.load(model_path)
        net.load_state_dict(checkpoint)
        net.eval()
        episode_count = MODEL_NUMBER
    else:
        episode_count = 0
    print("episode_count set to ", episode_count)




if not TRAINING:
    print([np.mean(episode_lengths), np.sqrt(np.var(episode_lengths)), np.mean(np.asarray(np.asarray(episode_lengths) < max_episode_length, dtype=float))])


Makes decisions and enacts them on the environment (env.step())

Asks the NN for an output (policy) given the current input (obs)

Records experience (state, action, reward) to update NN weights